In [1]:
from loguru import logger
import ndf.download
from ndf.datamining import datamining
import warnings
import sys
import pandas as pd

warnings.simplefilter(action='ignore', category=FutureWarning)

log_level = 'TRACE'
logger.add(sys.stderr, format="{time} {level} {message}", filter="my_module", level=log_level)
logger.add(sys.stdout, colorize=True, format="<green>{time}</green> <level>{message}</level>")
# logger.add("ndfdownload.log", rotation="30 MB", level=log_level)

2

In [2]:
import requests
import time

MAX_RETRIES = 4
TIME_TO_WAIT = 10

def download_manager(url):
    r = None
    # logger.info(f'Download settings: MAX_RETRIES:{MAX_RETRIES} | TIME_TO_WAIT:{TIME_TO_WAIT}')
    for _ in range(MAX_RETRIES):
        try:
            r = requests.get(url, verify=False)
        except requests.exceptions.RequestException as e:
            print(f'Download fail, {e}')
        if r:
            print(f'Download Ok')
            break
        else:
            wait = TIME_TO_WAIT * (_ + 1) if _ < 3 else TIME_TO_WAIT * (_ * 3)
            print(f'Download fail, retry in {wait} seconds...')
            time.sleep(wait)
    return r

In [3]:
# download_manager('https://www.tullettprebon.com/swap-execution-facility/daily-activity-summary.aspxa')

In [4]:
from datetime import datetime

a = datetime.now()
a.date()

datetime.date(2025, 6, 11)

In [5]:
# Logger settings

LOGGER_ENABLED = True

if not LOGGER_ENABLED:
    logger.remove()

In [6]:
d = ndf.download.download()
d.download_all()

2025-06-11 11:30:11.256 | INFO     | ndf.download:_download:55 - BGC - Download start >>>


2025-06-11T11:30:11.256404-0400 BGC - Download start >>>


2025-06-11 11:30:11.258 | DEBUG    | ndf.download:_download:58 - Try download bgc from http://dailyactprod.bgcsef.com/SEF/DailyAct/DailyAct_20250610.xlsx


2025-06-11T11:30:11.258056-0400 Try download bgc from http://dailyactprod.bgcsef.com/SEF/DailyAct/DailyAct_20250610.xlsx


2025-06-11 11:30:11.259 | INFO     | ndf.download:_download:98 - Downloading to /Users/cleiton/PycharmProjects/ndfutil/DATA/bgc_2025-06-11.xlsx from http://dailyactprod.bgcsef.com/SEF/DailyAct/DailyAct_20250610.xlsx


...

2025-06-11T11:37:15.328709-0400 Convert XLS to pandas dataframe


/Users/cleiton/PycharmProjects/ndfutil/venv/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
2025-06-11 11:37:15.604 | DEBUG    | ndf.download:_download:118 - Creating cache...


2025-06-11T11:37:15.604987-0400 Creating cache...


2025-06-11 11:37:15.607 | INFO     | ndf.download:_download:120 - gfi data success cached at /Users/cleiton/PycharmProjects/ndfutil/DATA/gfi_2025-06-11.pkl


2025-06-11T11:37:15.607168-0400 gfi data success cached at /Users/cleiton/PycharmProjects/ndfutil/DATA/gfi_2025-06-11.pkl
bgc ok
tradition ok
prebontullet ok
gfi ok


In [7]:
mining = datamining()

In [8]:
d.download_bgc()

2025-06-11 11:37:15.616 | INFO     | ndf.download:_download:55 - BGC - Download start >>>


2025-06-11T11:37:15.616032-0400 BGC - Download start >>>


2025-06-11 11:37:15.617 | DEBUG    | ndf.download:_download:58 - Try download bgc from http://dailyactprod.bgcsef.com/SEF/DailyAct/DailyAct_20250610.xlsx


2025-06-11T11:37:15.617496-0400 Try download bgc from http://dailyactprod.bgcsef.com/SEF/DailyAct/DailyAct_20250610.xlsx


2025-06-11 11:37:15.618 | INFO     | ndf.download:_download:98 - Downloading to /Users/cleiton/PycharmProjects/ndfutil/DATA/bgc_2025-06-11.xlsx from http://dailyactprod.bgcsef.com/SEF/DailyAct/DailyAct_20250610.xlsx


...

2025-06-11 11:41:46.314 | INFO     | ndf.download:download_manager:50 - Download fail, retry in 150 seconds...


2025-06-11T11:41:46.314145-0400 Download fail, retry in 150 seconds...


2025-06-11 11:44:16.317 | ERROR    | ndf.download:_download:125 - Download failed


2025-06-11T11:44:16.317033-0400 Download failed


True

In [9]:
import joblib

df = None
df = joblib.load('/Users/cleiton/PycharmProjects/ndfutil/DATA/tulletprebon_2025-05-24.pkl')

df


,Batch Date,Asset Class,Tradeable Instrument,Description,Opening Price,Trade High,Trade Low,Closing Price,Num of Trades,Total Notional Value
0,2025-05-23,CD,465636,S43 XOVER 5Y (SEF),315.25,315.25,315.25,315.25,1,10000000
1,2025-05-23,CD,466273,IG44 5Y - SEF,60.85,60.85,59.3,59.3,5,175000000
2,2025-05-23,CD,466928,HY44 5Y - SEF,105.55,105.55,105.06,105.29,17,175000000
3,2025-05-23,CD,CDX.EM.43.V1_60M,CDX.EM.43.V1_60M,96.79,96.79,96.63,96.65,8,85000000
4,2025-05-23,CD,CDX.EM.HY.43-V1_60M,CDX.EM.HY.43-V1_60M,92.68,92.68,92.68,92.68,1,10000000
...,...,...,...,...,...,...,...,...,...,...
503,2025-05-23,IR,THB.1Y.THOR.NDOIS_20260527,THB.1Y.THOR.NDOIS_20260527,0.0148,0.0148,0.0148,0.0148,1,1660000000
504,2025-05-23,IR,THB.6M.THOR.NDOIS ISDA2021_20251127,THB.6M.THOR.NDOIS ISDA2021_20251127,0.016025,0.016025,0.016025,0.016025,1,3290000000
505,2025-05-23,IR,THB.9M.THOR.NDOIS ISDA2021_20260227,THB.9M.THOR.NDOIS ISDA2021_20260227,0.015375,0.015375,0.015375,0.015375,1,4400000000
506,2025-05-23,IR,UF.2Y.SB_20270528,UF.2Y.SB_20270528,0.01285,0.01285,0.01285,0.01285,1,360000


In [10]:
# df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop([0], axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)
df.columns = df.iloc[0]
df.reset_index(drop=True, inplace=True)
df = df.rename(columns={'Contract Description': 'Description'})
df.drop(['Asset Class', 'Open', 'Low', 'High', 'Close', 'Block', 'Currency'], axis=1, inplace=True)
df = df[df['Volume'].notna()]
df

KeyError: "['Asset Class' 'Open' 'Low' 'High' 'Close' 'Block' 'Currency'] not found in axis"

In [ ]:
df

In [ ]:
#
# DADOS ANTIGOS 
#

import joblib

df_antigo = joblib.load('/Users/cleiton/PycharmProjects/ndfutil/DATA/BGC_2023-04-28.pkl')

df_antigo

In [ ]:
df_antigo.drop('Unnamed: 0', axis=1, inplace=True)
df_antigo

In [ ]:

df_antigo.drop([0, 1, 2], axis=0, inplace=True)
df_antigo.reset_index(drop=True, inplace=True)
df_antigo.columns = df_antigo.iloc[0]
df_antigo.reset_index(drop=True, inplace=True)
df_antigo = df_antigo.rename(columns={'Contract Description': 'Description'})
df_antigo.drop(['Asset Class', 'Open', 'Low', 'High', 'Close', 'Block', 'Currency'], axis=1, inplace=True)
df_antigo = df_antigo[df_antigo['Volume'].notna()]
df_antigo


In [ ]:
mining.bgc_calcs()

In [ ]:
mining.tulletprebon_calcs()

In [ ]:
mining.tradition_calcs()

In [ ]:
import joblib

df = None
df = joblib.load('/Users/cleiton/PycharmProjects/ndfutil/DATA/gfi_2025-03-08.pkl')

df


In [ ]:
df.drop([0], axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)
df.columns = df.iloc[0]
df.reset_index(drop=True, inplace=True)
df = df.rename(columns={'InstrumentDescription': 'Description'})
df.drop(['Asset Class', 'Open', 'Low', 'High', 'Close', 'Block', 'Currency'], axis=1, inplace=True)
df = df[df['Volume'].notna()]
df

In [ ]:
mining.gfi_calcs()

In [ ]:
tulletprebon['source'] = 'TulletPrebon'
tradition['source'] = 'Tradition'
bgc['source'] = 'bgc'
gfi['source'] = 'gfi'

In [ ]:
bgc

In [ ]:
df_result = pd.concat([tulletprebon, tradition, bgc, gfi])
df_result

In [ ]:
df_totals = df_result.query('Class == "TOTAL"')
df_totals

In [ ]:
df_totals.groupby(['Class']).sum()

In [ ]:
from numerize import numerize

market_total = df_totals.sum()['Volume']
market_total = numerize.numerize(float(market_total))
market_total